# Dots Parameter Playground

*This notebook is referred to as **Version 2** in other notebooks.*

This notebook is adapted from *Allen Cell Structure Segmenter* for segmentation of dot-like structures.
The purpose of this playground is to find the most appropriate parameters for a specific dataset to be segmented.


Key steps of the workflows:

* Min-max intensity normalization
* 2D Gaussian smoothing (slice-by-slice)
* 3D spot filter to detect dots
* Watershed for seperating falsely merged dots

J. Chen, L. Ding, M.P. Viana, M.C. Hendershott, R. Yang, I.A. Mueller, S.M. Rafelski. The Allen Cell Structure Segmenter: a new open source toolkit for segmenting 3D intracellular structures in fluorescence microscopy images. bioRxiv. 2018 Jan 1:491035.

<span style='color:red'> **1. Follow the red instruction lines through this notebook. Run the following two cells** </span>

In [ ]:
import numpy as np
import scipy
import skimage
import skimage.io
import nbimporter
#import Functions

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball, label     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label, regionprops, find_contours 
from scipy.ndimage import distance_transform_edt
from scipy import ndimage

In [ ]:
def ConvertToHyperstack(FILE_NAME, nbChannels, nbZplanes):
    
    raw_im = skimage.io.imread(FILE_NAME)
    
    # Find and parse dimensions of input image
    dims = raw_im.shape

    if len(dims) != 3:
        raise RuntimeError(f"Expected 3 dimensions, found {len(dims)}")

    x = dims[2]
    y = dims[1]
    zt = dims[0]

    # Calculate number of t frames (assume input image can have any number of time frames)
    t = int(zt/(nbZplanes*nbChannels))

    # Reshape to TZYXC
    im2 = raw_im.reshape((nbChannels,nbZplanes,t,y,x), order='F')
    # Move indices from CZTYX to TCZYX to match AICS format
    IMG = np.moveaxis(im2, [0, 1, 2, 3, 4], [1, 2, 0, 3, 4] )
    #IMG = np.moveaxis(im2, [0, 1, 2, 3, 4], [0, 2, 3, 4, 1])

    
    # Change type from uint16 to float32
    IMG = IMG.astype(np.float32)
    
    return IMG

## Loading the data
<span style='color:red'> **2. Change the variable `FILE_NAME`to the directory of one of the raw timelapse files.** </span>

<span style='color:red'> **3. Run the next two cells** </span>

In [ ]:
FILE_NAME = 'E:\\Weber Lab\\Practice Coding\\seg_batch_input\\42h_RNAi L4440 #2_WLW32_Timelapse_Bottom Half Embryo_25 planes_35 sec interval_Always alternate GFP, RFP_November 18 2022.tif'

IMG = ConvertToHyperstack(FILE_NAME, 2, 25)
print(IMG.shape) #(max nb of time pts, nb of channels, nb of planes, x, y)

In [ ]:
# Check input image
import matplotlib.pyplot as plt
plt.imshow(IMG[20,0,20,:,:])

<span style='color:red'> **4. Input the desired time point for analysis in place of `t1`, `t2` and `t3` and run all three cells** </span>

The timelapse file is made up of 3D stacks of an embryo over time. To select a specifc stack (timepoint) within the timelapse,
choose any three numbers between 0 and the max number of timepoints found two cells above.

In [ ]:
#####################
t1 = 15
structure_channel = 0
#####################

# first time frame for parameter search
struct_img1 = IMG[t1,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img1))

In [ ]:
t2 = 20

# second time frame for parameter search
struct_img2 = IMG[t2,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img2))

In [ ]:
t3 = 25
# third time frame for parameter search
struct_img3 = IMG[t3,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img3))

## Image segmentation

### Step 1: Pre-Processing

About selected algorithms and tuned parameters

* **Intensity normalization**: Parameter `intensity_scaling_param` has two options: two values, say `[A, B]`, or single value, say `[K]`. For the first case, `A` and `B` are non-negative values indicating that the full intensity range of the stack will first be cut-off into **[mean - A * std, mean + B * std]** and then rescaled to **[0, 1]**. The smaller the values of `A` and `B` are, the higher the contrast will be. For the second case, `K`>0 indicates min-max Normalization with an absolute intensity upper bound `K` (i.e., anything above `K` will be chopped off and reset as the minimum intensity of the stack) and `K`=0 means min-max Normalization without any intensity bound.

    * Parameter for Centrin-2:  `intensity_scaling_param = [8000]`
    * Parameter for Desmoplakin:  `intensity_scaling_param = [8000]`
    * Parameter for PMP34:  `intensity_scaling_param = [6000]`

These three all use Min-Max Normalization, just with slightly different absolute intensity upper bound. Using Min-Max normalization is meant to keep the full intensity profile near the brightest area, usually the center of the "balls" as in these structures. Such bright centers are important for the success of these workflows. Meanwhile, adding an absolute intensity upper bound is meant to be robust to imaging artifacts, like dead pixels. If there is one dead pixel in an image stack with extremely high intensity (way out of normal intensity range), the Min-Max normalization will squeeze the value of actual structures into a tiny range close to 0. So, we add an upper bound. For example, in Centrin-2, if a pixel has intensity value over 8000, this pixel will be treated as an outlier and reset as the minimum intensity of the stack. 

* **Smoothing** 

These three all use 2D gaussian smoothing with `gaussian_smoothing_sigma = 1`. Usually, 3D gaussian is used by default. But, here we apply 2D gaussian slice by slice, because when we do live imaging, these "ball"-shape structures could move visible amount (considering these "balls" themselves are tiny) during the time interval between two consecutive z slices. So, 3D guassian smoothing may further aggravate the subtle shift in consecutive z-slices. If your data do not have such problem, you can certainly try 3D gaussian by `image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)` with `gaussian_smoothing_sigma = 1`. To deal with very noisy data, you may consider increase `gaussian_smoothing_sigma` from 1 to a higher value, like 1.5 or 2.

<span style='color:red'> **5. Adjust `intensity_scaling_param` and `gaussian_smoothing_sigma` and run all 4 cells** <span>

In [ ]:
################################
intensity_scaling_param = [1,15]
gaussian_smoothing_sigma = 1
################################

# intensity normalization
struct_imgA = intensity_normalization(struct_img1.copy(), scaling_param=intensity_scaling_param)
struct_imgB = intensity_normalization(struct_img2.copy(), scaling_param=intensity_scaling_param)
struct_imgC = intensity_normalization(struct_img3.copy(), scaling_param=intensity_scaling_param)

# smoothing with gaussian filter
structure_img_smoothA = image_smoothing_gaussian_slice_by_slice(struct_imgA.copy(), sigma=gaussian_smoothing_sigma)
structure_img_smoothB = image_smoothing_gaussian_slice_by_slice(struct_imgB.copy(), sigma=gaussian_smoothing_sigma)
structure_img_smoothC = image_smoothing_gaussian_slice_by_slice(struct_imgC.copy(), sigma=gaussian_smoothing_sigma)

In [ ]:
# view the first image after smoothing
view(single_fluorescent_view(structure_img_smoothA))

In [ ]:
# view the second image after smoothing
view(single_fluorescent_view(structure_img_smoothB))

In [ ]:
# view the third image after smoothing
view(single_fluorescent_view(structure_img_smoothC))

### Step 2: Core Algorithm

#### step 2.1: Apply 3D Spot filter (S3)

* Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
    * `scale_x` is set based on the estimated radius of your target dots. For example, if visually the diameter of the dots is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have dots of very different sizes.  
    * `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf more dots and fatter segmentation, while larger `cutoff_x` could be less permisive and yield less dots and slimmer segmentation. 

* Parameter for Centrin-2:  `s3_param = [[1, 0.04]]`

* Parameter for Desmoplakin:  `s3_param = [[1, 0.012]]`

* Parameter for PMP34:  `s3_param = [[1,0.03]]`


<span style='color:red'> **6. Adjust `s3_param` as necessary and run all 4 cells** </span>

In [ ]:
################################
## PARAMETERS for this step ##
s3_param = [[0.9, 0.09]]
################################

bwA = dot_3d_wrapper(structure_img_smoothA.copy(), s3_param)
bwB = dot_3d_wrapper(structure_img_smoothB.copy(), s3_param)
bwC = dot_3d_wrapper(structure_img_smoothC.copy(), s3_param)

In [ ]:
# view the first segmentation result
view(seg_fluo_side_by_side(structure_img_smoothA,bwA,roi=['Full']))

In [ ]:
# view the second segmentation result
view(seg_fluo_side_by_side(structure_img_smoothB,bwB,roi=['Full']))

In [ ]:
# view the third segmentation result
view(seg_fluo_side_by_side(structure_img_smoothC,bwC,roi=['Full']))

##### Is the segmentation satisfactory? Here are some possible criteria:

--------------------------
* Note: next step (2.2) can further cut falsely merged spots.
* Is there any spot should be detected but not? Try to reduce `cutoff_x`
* Is there any spot should not be detected but actually appear in the result? Try to increase `cutoff_x` or try a larger `scale_x`
* Is the segmented size of the spots fatter than it should be? Try to increase `cutoff_x` or try a smaller `scale_x`
* Is there any spot that should be solid but segmented as a ring? Try to increase `scale_x`
* Are you observing spots with very different sizes? Try multiple sets of `scale_x` and `cutoff_x` 
--------------------------

#### If the results are satisfactory, go to Step 2.2 directly; otherwise, try to tweak the parameters based on the above suggestions. 

Assumption: the segmentation result is saved in a variable named `bw`.

#### Step 2.2: Watershed for cutting falsely merged spots

This step may take 2~3 minutes or longer, depending on the number of dots in your images. 

In general, no parameter tuning is needed in this step. The only parameter is `minArea=4`, which is the parameter for size thresholding in post-processing. In practice, when we know an object whose size is smaller than `minArea` will be removed eventually, we can just remove them before doing watershed to cut falsely merged cells. This is only meant to avoid unnecessary computation. 

<span style='color:red'> **Leave the cell below as is unless the WATERSHED function is desired** </span>

In [ ]:
# watershed
#minArea = 4
#Mask = remove_small_objects(bwA>0, min_size=minArea, connectivity=1, in_place=False) 
#Seed = dilation(peak_local_max(struct_imgA,labels=label(Mask), min_distance=2, indices=False), selem=ball(1))
#Watershed_Map = -1*distance_transform_edt(bwA)
#seg = watershed(Watershed_Map, label(Seed), mask=Mask, watershed_line=True)

#### Step 3: Post-Processing 

<span style='color:red'> **7. Designate the `minArea` size for segmented dot-like structures and run all 4 cells** </span>

(all structures < `minArea` will be removed)

In [ ]:
################################
## PARAMETERS for this step ##
minArea = 8
################################

segA = remove_small_objects(bwA.copy()>0, min_size=minArea, connectivity=1, in_place=False)
segB = remove_small_objects(bwB.copy()>0, min_size=minArea, connectivity=1, in_place=False)
segC = remove_small_objects(bwC.copy()>0, min_size=minArea, connectivity=1, in_place=False)

In [ ]:
# view the first post-processing result
view(seg_fluo_side_by_side(bwA, segA, roi=['Full']))

In [ ]:
# view the second post-processing result
view(seg_fluo_side_by_side(bwB, segB, roi=['Full']))

In [ ]:
# view the third post-processing result
view(seg_fluo_side_by_side(bwC, segC, roi=['Full']))

##  Result Inspection - Side-by-side followed by overlay

<span style='color:red'> **8. Visualize the side-by-side of the final segmented with the raw image for each 
    timepoint. Run the next 3 cells** </span>

In [ ]:
# first side-by-side result
view(seg_fluo_side_by_side(struct_img1, segA, roi=['Full']))

In [ ]:
# second side-by-side result
view(seg_fluo_side_by_side(struct_img2, segB, roi=['Full']))

In [ ]:
# third side-by-side result
view(seg_fluo_side_by_side(struct_img3, segC, roi=['Full']))

<span style='color:red'> **9. Visualize the overlay of the segmentation onto the raw image for a given slice in the stack. Change the variables `s1`, `s2` and `s3` to any number between 0 and 24 and run the next 4 cells** </span>

In [ ]:
#Overlay
rawA = struct_img1.copy()
rawB = struct_img2.copy()
rawC = struct_img3.copy()

for slice in range(segA.shape[0]):
    contoursApos = skimage.measure.find_contours(segA[slice,:,:]) #Finds positions where a contour lies of single slice of stack
    contoursBpos = skimage.measure.find_contours(segB[slice,:,:])
    contoursCpos = skimage.measure.find_contours(segC[slice,:,:])
    #print(contoursApos)
    #For each position found above, I want to change the pixel value to 0 in raw img 
    for objectA in contoursApos:
        for coordA in objectA:
            rawA[slice,int(coordA[0]), int(coordA[1])] = 255
    for objectB in contoursBpos:
        for coordB in objectB:
            rawB[slice,int(coordB[0]), int(coordB[1])] = 255
    for objectC in contoursCpos:
        for coordC in objectC:
            rawC[slice,int(coordC[0]), int(coordC[1])] = 255
    

In [ ]:
# first result 
s1 = 18 #slice number in the 3D stack
plt.imshow(rawA[s1,:,:])

In [ ]:
# second result
s2 = 18
plt.imshow(rawB[s2,:,:])

In [ ]:
# third result
s3 = 18
plt.imshow(rawC[s3,:,:])

### You may also physically save the segmentation results into a .tiff image
<span style='color:red'> **6. Name the output file and specify the output folder under `OmeTiffWriter`, and specify which `seg` file should be used for output** </span>


In [ ]:
segC = segC > 0
out=segC.astype(np.uint8)
out[out>0]=255
writer = OmeTiffWriter()
writer.save(out,'E:\\Weber Lab\\Imaging WLW32\\February 18, 2022\\Segmented\\Seg_TIME_32_43h_RNAi sun-1_WLW32_Timelapse_Bottom Half Embryo_20 planes_30 sec interval_Always alternate GFP, RFP_February 18 2021.ome.tiff')

In [ ]:
labelA = skimage.morphology.label(segA)
labelB = skimage.morphology.label(segB)
labelC = skimage.morphology.label(segC)
#labelC = skimage.morphology.label(segC, return_num=True)
#skimage.morphology.label(segA, return_num=True)

In [ ]:
out=labelA.astype(np.uint8)
writer = OmeTiffWriter()
writer.save(out,'E:\\Weber Lab\\Imaging WLW32\\43h_RNAi sun-1_WLW32_Timelapse_Bottom Half Embryo_20 planes_30 sec interval_Always alternate GFP, RFP_February 18 202101.tiff')

In [ ]:
#view(single_fluorescent_view(labelA))
#print(labelA.max())
scipy.ndimage.find_objects(labelA)

In [ ]:
view(single_fluorescent_view(labelB))
#print(labelB.max())

In [ ]:
view(single_fluorescent_view(labelC))
#print(labelB.max())